In [1]:
import sys  
sys.path.append("../../")
from LIB.utils import * 
from datetime import timedelta

In [2]:
train_df=pd.read_csv("./final_data/proceed_train.csv")
train_df.date=pd.to_datetime(train_df.date)
test=pd.read_csv("./final_data/proceed_test.csv")
test.date=pd.to_datetime(test.date)


In [3]:
test["train"]=0
train_df["train"]=1
data=pd.concat([test,train_df])
data.sort_values(["travel_from","date"],inplace=True,ascending=True)
data["diff_btw_0_1_next_bus"]=(data["date"]-data.groupby(["travel_from"]).date.shift(-1)).dt.total_seconds()/60/60
data["diff_btw_0_1_previous_bus"]=(data["date"]-data.groupby(["travel_from"]).date.shift(1)).dt.total_seconds()/60/60
data["diff_btw_0_2_next_bus"]=(data["date"]-data.groupby(["travel_from"]).date.shift(-2)).dt.total_seconds()/60/60
data["diff_btw_0_2_previous_bus"]=(data["date"]-data.groupby(["travel_from"]).date.shift(2)).dt.total_seconds()/60/60
data["diff_btw_0_3_next_bus"]=(data["date"]-data.groupby(["travel_from"]).date.shift(-3)).dt.total_seconds()/60/60
data["diff_btw_0_3_previous_bus"]=(data["date"]-data.groupby(["travel_from"]).date.shift(3)).dt.total_seconds()/60/60
data["diff_btw_next_previous_bus"]=(data.groupby(["travel_from"]).date.shift(-1)-data.groupby(["travel_from"]).date.shift(1)).dt.total_seconds()/60/60
fea=['diff_btw_0_1_next_bus', 'diff_btw_0_1_previous_bus', 'diff_btw_0_2_next_bus', 'diff_btw_0_2_previous_bus',
     'diff_btw_0_3_next_bus', 'diff_btw_0_3_previous_bus',
     'diff_btw_next_previous_bus']
data[fea]=data.groupby(["travel_from"])[fea].fillna(method="ffill")
data[fea]=data.groupby(["travel_from"])[fea].fillna(method="backfill")

for i in [1,3,5,7,10,15] :
    data["count_trip_per_{}min_travel_from".format(str(i))]=data.groupby([pd.Grouper(key="date",freq='{}min'.format(i)),"travel_from"]).ride_id.transform("count")
    data["count_trip_per_{}min".format(str(i))]=data.groupby([pd.Grouper(key="date",freq='{}min'.format(i))]).ride_id.transform("count")

In [7]:
holidays_date=[datetime(2018,1,1),
              datetime(2018,3,30),
              datetime(2018,4,2),
              datetime(2018,6,1),
              datetime(2018,6,15),
              datetime(2017,10,20),
              datetime(2017,11,7),
              datetime(2017,12,12),
              datetime(2017,12,25),
              datetime(2017,12,26)]
                                      
holidays_date_mins_one_day=[ (day- timedelta(days=1)).date() for day in holidays_date]
holidays_date_mins_two_day=[ (day- timedelta(days=2)).date() for day in holidays_date]

holidays_date_plus_one_day=[ (day+ timedelta(days=1)).date() for day in holidays_date]
holidays_date=[ day.date() for day in holidays_date]

data["is_holidays"]=0
data["is_after_toomorw_holidays"]=0


data.loc[data.date.dt.date.isin(holidays_date),"is_holidays"]=1
data.loc[data.date.dt.date.isin(holidays_date_mins_two_day),"is_after_toomorw_holidays"]=1

In [8]:
distance = {'Migori': 372 , 'Keroka': 281, 'Homa Bay':364, 'Kisii':305.1, 'Keumbu':295, 'Rongo':332,
'Kijauri':271, 'Oyugis':330.6, 'Awendo':351, 'Sirare':392, 'Nyachenge':326, 'Kehancha': 387.7,
'Kendu Bay':347, 'Sori':399, 'Rodi':348, 'Mbita':401, 'Ndhiwa': 371}
data["travel_from_distance"]=data.travel_from.map(distance)

In [9]:
time = {'Migori': 7*60+8 , 'Keroka': 5*60, 'Homa Bay':7*60, 'Kisii':5*60+34, 'Keumbu':5*60+20, 'Rongo':6*60+21,
'Kijauri':60*4+50,'Oyugis':5*60+50, 'Awendo':6*60+38, 'Sirare':7*60+30, 'Nyachenge':6*60+10, 'Kehancha':7*60+10,
'Kendu Bay':6*60+10, 'Sori':7*60+30, 'Rodi':6*60+40, 'Mbita':7*60+23, 'Ndhiwa': 7*60}
data["travel_from_time"]=data.travel_from.map(time)

In [10]:
data["Speed"]=data.travel_from_time/data.travel_from_distance


In [11]:
from datetime import datetime, timedelta
time = {'Migori': 7*60+8 , 'Keroka': 5*60, 'Homa Bay':7*60, 'Kisii':5*60+34, 'Keumbu':5*60+20, 'Rongo':6*60+21,
'Kijauri':60*4+50,'Oyugis':5*60+50, 'Awendo':6*60+38, 'Sirare':7*60+30, 'Nyachenge':6*60+10, 'Kehancha':7*60+10,
'Kendu Bay':6*60+10, 'Sori':7*60+30, 'Rodi':6*60+40, 'Mbita':7*60+23, 'Ndhiwa': 7*60}
for key in time.keys(): 
    time[key]=timedelta( minutes=time[key])
data["travel_from_date"]=data.travel_from.map(time)
data["arrived_date"]=data.date+ data.travel_from_date
data["hod_arrived_date"]=data["arrived_date"].dt.hour
data["minute_arrived_date"]=data["arrived_date"].dt.minute
del data["travel_from_date"],data["arrived_date"]
data["is_rush_hour"]=0
data.loc[data.hod_arrived_date.between(7,17),"is_rush_hour"]=1

In [12]:
external_data=pd.read_csv("./final_data/weather_mean.csv")

external_data["date"]=pd.to_datetime(external_data.date).dt.date
data["12H"]=data["hour"].apply(lambda x : 1 if 0<=x<12 else  0 )
data.date=data.date.dt.date
data=data.merge(external_data,how="left",on=["12H","date"])

In [13]:
del data["year_woy"]

In [14]:
def haversine(latlong1, latlong2):
    lat_1s = np.array([x[0] for x in latlong1])
    lon_1s = np.array([x[1] for x in latlong1])
    lat_2s = np.array([x[0] for x in latlong2])
    lon_2s = np.array([x[1] for x in latlong2])
    lon1, lat1, lon2, lat2 = map(np.radians, [lon_1s, lat_1s, lon_2s, lat_2s])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    km = 6367 * 2 * np.arcsin(np.sqrt(a))
    return km



def bearing(latlong1, latlong2):
    lat1 = np.array([x[0] for x in latlong1])
    lon1 = np.array([x[1] for x in latlong1])
    lat2 = np.array([x[0] for x in latlong2])
    lon2 = np.array([x[1] for x in latlong2])
    delta_lon = np.radians(lon2 - lon1)
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    x = np.cos(lat2) * np.sin(delta_lon)
    y = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(delta_lon)
    return np.degrees(np.arctan2(x, y))

In [15]:
lat = {'Migori': -1.063590 , 'Keroka': -0.775880, 'Homa Bay':-0.527180, 'Kisii':-0.674230, 'Keumbu':-0.734310, 'Rongo':-0.755324,
'Kijauri':-0.772980, 'Oyugis':-0.509360, 'Awendo':-0.893820, 'Sirare':-1.246610, 'Nyachenge':-0.726110, 'Kehancha': -1.192040,
'Kendu Bay':-0.369200, 'Sori':-0.843329, 'Rodi':-0.624863, 'Mbita':-0.421886, 'Ndhiwa': -0.733300}

Long = {'Migori': 34.472919 , 'Keroka': 34.945740, 'Homa Bay':34.455711, 'Kisii':34.770741, 'Keumbu':34.846291, 'Rongo':34.599899,
'Kijauri':34.967369, 'Oyugis':34.735409, 'Awendo':34.530819, 'Sirare':34.475368, 'Nyachenge':34.648750, 'Kehancha': 34.621380,
'Kendu Bay':34.652802, 'Sori':34.158842, 'Rodi':34.502402, 'Mbita':34.207587, 'Ndhiwa': 34.366600}
data["travel_from_lat"]=data.travel_from.map(lat)
data["travel_from_Long"]=data.travel_from.map(Long)
data["Afya_Centr_Long"]=36.827845
data["Afya_Centr_lat"]=-1.287811
data["haversine_distance"]=haversine(data[['travel_from_lat', 'travel_from_Long']].apply(tuple, axis=1),
                                    data[['Afya_Centr_lat', 'Afya_Centr_Long']].apply(tuple, axis=1))

data["bearing_distance"]=bearing(data[['travel_from_lat', 'travel_from_Long']].apply(tuple, axis=1),
                                    data[['Afya_Centr_lat', 'Afya_Centr_Long']].apply(tuple, axis=1))
del data["Afya_Centr_Long"],data["Afya_Centr_lat"]

In [16]:
train_df=data[data.train==1]
test_df=data[data.train==0]
test_df.drop(["number_of_ticket"],axis=1,inplace=True)
map_categorical_feautres(Data=[train_df,test_df],columns=["car_type","travel_from","year"])  
uber=pd.read_csv("./final_data/uber_data.csv")
train_df=train_df.merge(uber,how="left",on="ride_id")
test_df=test_df.merge(uber,how="left",on="ride_id")

car_type
travel_from
year


In [17]:
col_to_aggr=[
            ["hour"],
            ["travel_from"],
            ["minute"] ,
            ["hour","dayofweek"],
            ["hour","travel_from"],
            ["dayofweek","travel_from"] ,
            ["minute","travel_from"] 
 ]
train_df,test_df=aggr_func(train=train_df,test=test_df,vars_to_agg=col_to_aggr,func=["mean"],
         vars_be_agg="number_of_ticket",fillnan=True,bagging=False,ID="ride_id" ,n_folds=6,shuffle=True)


In [18]:
col_to_aggr=["travel_from"]
train_df,test_df=aggr_func(train=train_df,test=test_df,vars_to_agg=col_to_aggr,func=["sum"],
         vars_be_agg="number_of_ticket",fillnan=True,bagging=False,ID="ride_id" ,n_folds=6,shuffle=True)

In [19]:
train_df.number_of_ticket_by_travel_from_sum=train_df.number_of_ticket_by_travel_from_sum/train_df.number_of_ticket.sum()
test_df.number_of_ticket_by_travel_from_sum=test_df.number_of_ticket_by_travel_from_sum/train_df.number_of_ticket.sum()


In [22]:
col_to_remove=[ 'bearing_distance', 'car_type',
                      'dayofmonth', 'dayofweek', 'dayofyear', 'diff_btw_0_3_previous_bus', 
                      'geometric_mean_travel_time_hod_arrved_date', 'geometric_standard_deviation_travel_time_dow', 
                      'geometric_standard_deviation_travel_time_hod_after_8h', 'geometric_standard_deviation_travel_time_hod_after_9h', 
                      'geometric_standard_deviation_travel_time_hod_arrved_date', 'geometric_standard_deviation_travel_time_month', 
                      'hod_arrived_date', 'hour', 'max_capacity', 'mean_travel_time_hod_arrved_date', 
                      'minute_arrived_date', 'month', 'quarter', 
                      'standard_deviation_travel_time_dow', 'standard_deviation_travel_time_hod_after_8h', 
                      'standard_deviation_travel_time_hod_after_9h', 'standard_deviation_travel_time_hod_arrved_date', 
                      'standard_deviation_travel_time_month', 'train', 'travel_date', 
                      'travel_from', 'travel_from_Long', 'travel_from_lat', 'travel_time']

In [23]:
v="final"

train_df.drop(col_to_remove,axis=1).to_csv("./final_data/train_{}.csv".format(v),index=False)
test_df.drop(col_to_remove,axis=1).to_csv("./final_data/test_{}.csv".format(v),index=False)